# Présentation des données

:::{image} car-cover.jpg
:::

## Introduction

### 1. Contexte et problématique métier

Dans le secteur de la vente automobile, et particulièrement pour les concessionnaires achetant des véhicules aux enchères, l'estimation du prix de revente est critique. L'objectif est de déterminer un prix d'achat maximal qui garantisse une marge bénéficiaire.

Cependant, se baser sur une simple estimation moyenne (le prix "moyen" du marché) est une stratégie risquée. Si le concessionnaire achète un véhicule à son prix moyen estimé, il court le risque de perdre de l'argent dans 50 % des cas (si la voiture se revend finalement moins cher).

L'objectif de ce projet est de sécuriser les achats en passant d'une prédiction ponctuelle à une **quantification d'incertitude**. Le concessionnaire ne cherche pas le prix "juste", mais un **prix plancher** : "Je veux être sûr à 90 % de pouvoir revendre ce véhicule au moins X €".

### 2. Présentation des données

Pour cette étude, nous utilisons le jeu de données **Car Prices Dataset** (disponible sur [Kaggle](https://www.kaggle.com/datasets/sidharth178/car-prices-dataset)).
Il s'agit d'un registre de voitures d'occasion contenant des caractéristiques techniques et commerciales telles que :
* La marque et le modèle.
* L'année de fabrication et le kilométrage.
* Le type de carburant et l'état général.

La tâche est une **régression** visant à prédire le prix de vente (variable continue). Ce jeu de données est idéal pour la quantification d'incertitude car il présente une forte **hétéroscédasticité** : la variance du prix n'est pas constante.

### 3. Approche méthodologique : Régression quantile et prédiction conforme

Les modèles de régression classiques minimisent l'erreur moyenne. Ils ne répondent pas à notre besoin de garantie.

Pour pallier ce manque, nous intégrons plusieurs méthodes issues de la **prédiction conforme**. Ces méthodes transforment la prédiction simple en intervalles de prédictions avec une garantie statistique de fiabilité.

Cette approche permet au métier de fixer ses prix d'achat sur des garanties statistiques (ex: "Je suis sûr à 80% de revendre au moins 14 000€") plutôt que sur des intuitions.

### 4. Applicabilité de la prédiction conforme

Pour bénéficier des garanties théoriques de la prédiction conforme, il faut que les données respectent l'hypothèse d'**échangeabilité**. Dans le contexte du marché automobile, cette hypothèse est raisonnable car les véhicules sont vendus dans différentes régions et conditions, et les données sont réparties aléatoirement entre les ensembles d'entraînement, de calibration et de test.

De plus, ce registre a été collecté dans un intervalle de temps réduit, ce qui permet en pratique de valider l'hypothèse d'échangeabilité nécessaire aux méthodes de prédiction conforme.

## Analyse exploratoire des données

### Chargement des données

In [86]:
from kagglehub import KaggleDatasetAdapter, dataset_load
import polars as pl

# Polars display options
pl.Config.set_tbl_hide_dataframe_shape(True)
pl.Config.set_float_precision(2)

df: pl.DataFrame = dataset_load(
    adapter=KaggleDatasetAdapter.POLARS,
    handle="sidharth178/car-prices-dataset",
    path="train.csv",
).collect()
df.head()

ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
i64,i64,str,str,str,i64,str,str,str,str,str,f64,str,str,str,str,str,i64
45654403,13328,"""1399""","""LEXUS""","""RX 450""",2010,"""Jeep""","""Yes""","""Hybrid""","""3.5""","""186005 km""",6.00,"""Automatic""","""4x4""","""04-May""","""Left wheel""","""Silver""",12
44731507,16621,"""1018""","""CHEVROLET""","""Equinox""",2011,"""Jeep""","""No""","""Petrol""","""3""","""192000 km""",6.00,"""Tiptronic""","""4x4""","""04-May""","""Left wheel""","""Black""",8
45774419,8467,"""-""","""HONDA""","""FIT""",2006,"""Hatchback""","""No""","""Petrol""","""1.3""","""200000 km""",4.00,"""Variator""","""Front""","""04-May""","""Right-hand drive""","""Black""",2
45769185,3607,"""862""","""FORD""","""Escape""",2011,"""Jeep""","""Yes""","""Hybrid""","""2.5""","""168966 km""",4.00,"""Automatic""","""4x4""","""04-May""","""Left wheel""","""White""",0
45809263,11726,"""446""","""HONDA""","""FIT""",2014,"""Hatchback""","""Yes""","""Petrol""","""1.3""","""91901 km""",4.00,"""Automatic""","""Front""","""04-May""","""Left wheel""","""Silver""",4


### Nettoyage des données

In [87]:
df = df.drop(["ID"])

# Extract features and cleaning
df = df.with_columns(
    # Create Turbo binary feature
    pl.col("Engine volume").str.contains("Turbo").alias("Turbo"),
    # Parse Engine volume: extract first number (e.g., '2.5 Turbo' -> 2.5)
    pl.col("Engine volume").str.extract(r"(\d+\.?\d*)", 1).cast(pl.Float64),
    # Parse Mileage: remove 'km' and convert to int
    pl.col("Mileage").str.replace(" km", "").cast(pl.Int64),
    # Cast Levy to numeric ('-' becomes null)
    pl.col("Levy").cast(pl.Int64, strict=False),
    # Convert Leather interior to binary
    (pl.col("Leather interior") == "Yes").cast(pl.Boolean),
    # Parse Doors: extract first number
    pl.col("Doors").str.extract(r"(\d+)", 1).cast(pl.Int64),
)

# Rename columns
df = df.rename(
    {
        "Engine volume": "Engine volume (L)",
        "Mileage": "Mileage (km)",
        "Prod. year": "Production year",
        "Levy": "Levy tax",
        "Manufacturer": "Brand",
    }
)
df.head()

Price,Levy tax,Brand,Model,Production year,Category,Leather interior,Fuel type,Engine volume (L),Mileage (km),Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Turbo
i64,i64,str,str,i64,str,bool,str,f64,i64,f64,str,str,i64,str,str,i64,bool
13328,1399,"""LEXUS""","""RX 450""",2010,"""Jeep""",true,"""Hybrid""",3.50,186005,6.00,"""Automatic""","""4x4""",4,"""Left wheel""","""Silver""",12,false
16621,1018,"""CHEVROLET""","""Equinox""",2011,"""Jeep""",false,"""Petrol""",3.00,192000,6.00,"""Tiptronic""","""4x4""",4,"""Left wheel""","""Black""",8,false
8467,null,"""HONDA""","""FIT""",2006,"""Hatchback""",false,"""Petrol""",1.30,200000,4.00,"""Variator""","""Front""",4,"""Right-hand drive""","""Black""",2,false
3607,862,"""FORD""","""Escape""",2011,"""Jeep""",true,"""Hybrid""",2.50,168966,4.00,"""Automatic""","""4x4""",4,"""Left wheel""","""White""",0,false
11726,446,"""HONDA""","""FIT""",2014,"""Hatchback""",true,"""Petrol""",1.30,91901,4.00,"""Automatic""","""Front""",4,"""Left wheel""","""Silver""",4,false


### Statistiques descriptives

In [88]:
df.describe()

statistic,Price,Levy tax,Brand,Model,Production year,Category,Leather interior,Fuel type,Engine volume (L),Mileage (km),Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Turbo
str,f64,f64,str,str,f64,str,f64,str,f64,f64,f64,str,str,f64,str,str,f64,f64
"""count""",19237.00,13418.00,"""19237""","""19237""",19237.00,"""19237""",19237.00,"""19237""",19237.00,19237.00,19237.00,"""19237""","""19237""",19237.00,"""19237""","""19237""",19237.00,19237.00
"""null_count""",0.00,5819.00,"""0""","""0""",0.00,"""0""",0.00,"""0""",0.00,0.00,0.00,"""0""","""0""",0.00,"""0""","""0""",0.00,0.00
"""mean""",18555.93,906.84,null,null,2010.91,null,0.73,null,2.31,1532235.69,4.58,null,null,3.93,null,null,6.58,0.10
"""std""",190581.27,461.87,null,null,5.67,null,null,null,0.88,48403869.38,1.20,null,null,0.40,null,null,4.32,null
"""min""",1.00,87.00,"""ACURA""","""09-Mar""",1939.00,"""Cabriolet""",0.00,"""CNG""",0.00,0.00,1.00,"""Automatic""","""4x4""",2.00,"""Left wheel""","""Beige""",0.00,0.00
"""25%""",5331.00,640.00,null,null,2009.00,null,null,null,1.80,70139.00,4.00,null,null,4.00,null,null,4.00,null
"""50%""",13172.00,781.00,null,null,2012.00,null,null,null,2.00,126000.00,4.00,null,null,4.00,null,null,6.00,null
"""75%""",22075.00,1058.00,null,null,2015.00,null,null,null,2.50,188888.00,4.00,null,null,4.00,null,null,12.00,null
"""max""",26307500.00,11714.00,"""სხვა""","""xD""",2020.00,"""Universal""",1.00,"""Plug-in Hybrid""",20.00,2147483647.00,16.00,"""Variator""","""Rear""",5.00,"""Right-hand drive""","""Yellow""",16.00,1.00


Il y a des valeurs aberrantes dans les données pour le prix et le kilométrage.

### Nettoyage des données aberrantes

In [89]:
df.sort("Price", descending=True).head(3)

Price,Levy tax,Brand,Model,Production year,Category,Leather interior,Fuel type,Engine volume (L),Mileage (km),Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Turbo
i64,i64,str,str,i64,str,bool,str,f64,i64,f64,str,str,i64,str,str,i64,bool
26307500,null,"""OPEL""","""Combo""",1999,"""Goods wagon""",false,"""Diesel""",1.70,99999,4.00,"""Manual""","""Front""",2,"""Left wheel""","""Blue""",0,false
872946,2067,"""LAMBORGHINI""","""Urus""",2019,"""Universal""",true,"""Petrol""",4.00,2531,8.00,"""Tiptronic""","""4x4""",4,"""Left wheel""","""Black""",0,false
627220,null,"""MERCEDES-BENZ""","""G 65 AMG 63AMG""",2020,"""Jeep""",true,"""Petrol""",6.30,0,8.00,"""Tiptronic""","""4x4""",4,"""Left wheel""","""Black""",12,true


In [90]:
df.sort("Mileage (km)", descending=True).head(8)

Price,Levy tax,Brand,Model,Production year,Category,Leather interior,Fuel type,Engine volume (L),Mileage (km),Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Turbo
i64,i64,str,str,i64,str,bool,str,f64,i64,f64,str,str,i64,str,str,i64,bool
6899,null,"""VOLKSWAGEN""","""Golf""",1999,"""Sedan""",false,"""Petrol""",1.60,2147483647,4.00,"""Manual""","""Front""",4,"""Left wheel""","""Black""",4,false
5959,null,"""MERCEDES-BENZ""","""C 180""",1995,"""Sedan""",false,"""CNG""",1.80,2147483647,4.00,"""Manual""","""Rear""",4,"""Left wheel""","""Blue""",5,false
10036,null,"""SUBARU""","""Forester""",2005,"""Jeep""",false,"""Petrol""",2.00,2147483647,4.00,"""Tiptronic""","""4x4""",4,"""Right-hand drive""","""White""",12,false
2200,null,"""UAZ""","""31514""",1968,"""Jeep""",true,"""CNG""",2.40,2147483647,4.00,"""Manual""","""4x4""",4,"""Left wheel""","""Black""",10,false
3,null,"""BMW""","""525""",1995,"""Sedan""",false,"""Petrol""",2.80,2147483647,6.00,"""Manual""","""Rear""",4,"""Left wheel""","""Black""",3,false
15681,null,"""TOYOTA""","""Prius""",2008,"""Sedan""",false,"""Petrol""",2.00,2147483647,4.00,"""Automatic""","""Front""",4,"""Left wheel""","""Blue""",0,false
18817,1995,"""FORD""","""Transit""",2003,"""Microbus""",false,"""Diesel""",2.40,2147483647,4.00,"""Manual""","""Front""",2,"""Left wheel""","""White""",2,true
4234,null,"""MERCEDES-BENZ""","""E 200""",2001,"""Sedan""",true,"""CNG""",2.60,1777777778,6.00,"""Tiptronic""","""Rear""",4,"""Right-hand drive""","""Black""",10,false


In [91]:
# On supprime les valeurs aberrantes

condition = (pl.col("Price") < 1_000_000) & (pl.col("Mileage (km)") < 1_000_000)
df = df.filter(condition)

### Distribution des variables numériques

In [92]:
import altair as alt

alt.data_transformers.enable("vegafusion")

numerical_cols = [
    "Production year",
    "Mileage (km)",
    "Price",
    "Levy tax",
    "Engine volume (L)",
]

for col in numerical_cols:
    alt.Chart(df).mark_boxplot(outliers={"size": 5}).encode(
        alt.X(f"{col}:Q").scale(zero=False),
    ).properties(title=f"Distribution of {col}").show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

### Distribution des variables catégorielles et booléennes

In [93]:
low_card_categorical = df.select([s for s in df if s.n_unique() < 50]).columns

for col in low_card_categorical:
    alt.Chart(df).mark_bar().encode(
        alt.X("count()"),
        alt.Y(f"{col}:N", sort="-x", title=None),
        tooltip=[f"{col}:N", "count()"],
    ).properties(
        title=f"Distribution of {col}",
    ).show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

### Matrice de corrélation

On observe une faible corrélation entre le prix de la voiture et l'année de production, le kilométrage et la présence d'un turbo.

In [94]:
from utils import plot_correlation

plot_correlation(df)

alt.ConcatChart(...)

### Évolution des prix par année de production

In [95]:
alt.Chart(df).mark_bar().encode(
    alt.X("Production year:T"),
    alt.Y("mean(Price):Q", title=None),
    tooltip=["Production year:T", "mean(Price):Q"],
).properties(title="Average Price by Production Year")

alt.Chart(...)

### Relation entre prix et année de production

In [96]:
alt.Chart(df).mark_rect(clip=True).encode(
    alt.X("Production year:Q").bin(maxbins=50),
    alt.Y("Price:Q").bin(maxbins=200).scale(domain=[0, 200_000]),
    alt.Color("count()").scale(type="log"),
    tooltip=["count()"],
).properties(title="Price vs Production year (Density Plot)")

alt.Chart(...)

### Relation entre prix et kilométrage

In [97]:
alt.Chart(df).mark_rect(clip=True).encode(
    alt.X("Mileage (km):Q").bin(maxbins=50).scale(domain=[0, 500_000]),
    alt.Y("Price:Q").bin(maxbins=100).scale(domain=[0, 200_000]),
    alt.Color("count()").scale(type="log"),
    tooltip=["count()"],
).properties(title="Price vs Mileage (Density Plot)")

alt.Chart(...)

### Distribution des prix par type de carburant

In [98]:
alt.Chart(df).mark_boxplot(extent="min-max", clip=True).encode(
    alt.X("Price:Q").scale(domain=[0, 50_000]),
    alt.Y("Fuel type:N", title=None),
    alt.Color("Fuel type:N", legend=None),
).properties(title="Price Distribution by Fuel Type")

alt.Chart(...)

### Distribution des prix par catégorie de véhicule

In [99]:
alt.Chart(df).mark_boxplot(extent="min-max", clip=True).encode(
    alt.X("Price:Q").scale(domain=[0, 50_000]),
    alt.Y("Category:N", title=None),
    alt.Color("Category:N", legend=None),
).properties(title="Price Distribution by Vehicle Category")

alt.Chart(...)

### Distribution des prix par top 10 marque

In [100]:
# Get top 10 brands by count
top_brands = df["Brand"].value_counts(sort=True).head(10)["Brand"].to_list()
df_top_brands = df.filter(pl.col("Brand").is_in(top_brands))

alt.Chart(df_top_brands).mark_boxplot(extent="min-max", clip=True).encode(
    alt.X("Price:Q").scale(domain=[0, 50_000]),
    alt.Y("Brand:N", title=None, sort="-x"),
    alt.Color("Brand:N", legend=None),
).properties(title="Price Distribution by Top 10 Brands")

alt.Chart(...)

On constate une hétéroscédasticité des prix en fonction du kilométrage des voitures, des marques, des catégories de véhicules et des types de carburant.

### Conversion en données catégorielles

Utile plus tard pour la gestion automatique des variables catégorielles dans les modèles.

In [101]:
categorical_features = [
    "Brand",
    "Category",
    "Fuel type",
    "Gear box type",
    "Drive wheels",
    "Wheel",
    "Color",
]
# Cast to categorical dtypes
df = df.cast({col: pl.Categorical for col in categorical_features})

## Sauvegarde des données nettoyées

In [ ]:
# Save preprocessed data for modeling
df.write_parquet("../../data/car_prices_clean.parquet")